In [1]:
# Imports
import pandas as pd
import xpress as xp

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: C:\Users\lrr1kor\AppData\Local\Programs\Python\Python39\Lib\site-packages\xpress\license\community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


In [2]:
# Data processing here

## Input Section
def read_excel():
    excel_file = pd.ExcelFile("data/input_data.xlsx")
    input_df_dict = {sheet_name: excel_file.parse(sheet_name)
                     for sheet_name in excel_file.sheet_names}
    return input_df_dict

def load_data():
    return get_modified_data(read_excel())

def get_modified_data(input_df_dict):
    input_param_dict = input_df_dict['parameters'].set_index('attribute')['value'].to_dict()
    return input_df_dict, input_param_dict

# Output Section
def _create_outputs_df(opt_series, cols, name, output_df_dict):
    df = pd.DataFrame(data=opt_series, index=opt_series.index.values).reset_index()
    df.columns = cols
    output_df_dict[name] = df

def write_outputs_xpress(dict_of_variables, model):
    output_df_dict = {}
    cols = ['month', 'value']
    for name, var in dict_of_variables.items():
        opt_series = pd.Series({k + 1: model.getSolution(v) for k, v in var.items()})
        _create_outputs_df(opt_series, cols, name, output_df_dict)
    return output_df_dict

def write_to_csv(output_df_dict):
    for key, df in output_df_dict.items():
        out_name = ''.join(('optimal_', key, '.csv'))
        out_name = 'data/' + out_name
        df.to_csv(out_name, index = False)

In [3]:
#
# Main Program
#
# ================== Set up data ==================
# Reading both sheets of input excel file and putting in dataframe 
input_df_dict, input_param_dict = load_data()

# ================== Set up the optimization model ==================

model = xp.problem(name='multiperiod_prod_planning')

# ================== Decision variables ==================
production_variables = xp.vars(input_df_dict['input_data'].index, name='X', vartype=xp.continuous)
overtime_prod_variables = xp.vars(input_df_dict['input_data'].index, name='Y', vartype=xp.continuous)
inventory_variables = xp.vars(input_df_dict['input_data'].index, name='I', vartype=xp.continuous)

model.addVariable(production_variables, overtime_prod_variables, inventory_variables)

# ================== Inventory balance constraints ==================
model.addConstraint(
    xp.constraint(
        body=inventory_variables[period - 1] + production_variables[period] + overtime_prod_variables[period] - inventory_variables[period],
        sense=xp.eq,
        name='inv_balance' + str(period),
        rhs=value.demand)
    for period, value in input_df_dict['input_data'].iloc[1:].iterrows())

# inv balance for first period
model.addConstraint(
    xp.constraint(
        body=production_variables[0] + overtime_prod_variables[0] - inventory_variables[0],
        sense=xp.eq,
        name='inv_balance0',
        rhs=input_df_dict['input_data'].iloc[0].demand - input_param_dict['initial_inventory']))

# ================== Production capacity constraints ==================
model.addConstraint(
    xp.constraint(
        body=value,
        sense=xp.leq,
        name='prod_cap_month_' + str(index),
        rhs=input_df_dict['input_data'].iloc[index].regular_production_capacity)
    for index, value in production_variables.items())

# ================== Overtime Production capacity constraints ==================
model.addConstraint(
    xp.constraint(
        body=value,
        sense=xp.leq,
        name='over_prod_cap_month_' + str(index),
        rhs=input_df_dict['input_data'].iloc[index].overtime_prod_capacity)
    for index, value in overtime_prod_variables.items())


# ================== Costs and objective function ==================
total_holding_cost = input_param_dict['holding_cost'] * xp.Sum(inventory_variables)
total_production_cost = 40 * xp.Sum(production_variables)
total_overtime_prod_cost = 45 * xp.Sum(overtime_prod_variables)

objective = total_holding_cost + total_production_cost + total_overtime_prod_cost

model.setObjective(objective, sense=xp.minimize)

# ================== Optimization ==================

model.write(model.name(), 'lp')

model.solve()

# for LP: 1: optimal, 2: infeasible, 5: unbounded. for MIP: 5: infeasible, 6: optimal, 7: unbounded
if model.getProbStatus() == 1:
    print(f'The solution is optimal and the objective value is ${model.getObjVal():,.2f}!')

# ================== Output ==================
dict_of_variables = {'production_variables': production_variables,
                     'overtime_prod_variables': overtime_prod_variables,
                     'inventory_variables': inventory_variables}

output_df = write_outputs_xpress(dict_of_variables, model)
write_to_csv(output_df)

FICO Xpress v8.11.3, Community, solve started 11:30:44, Jul 13, 2021
Heap usage: 345KB (peak 345KB, 538KB system)
Minimizing LP multiperiod_prod_planning with these control settings:
OUTPUTLOG = 1
Original problem has:
        18 rows           18 cols           35 elements
Presolved problem has:
         5 rows           16 cols           20 elements
Presolve finished in 0 seconds
Heap usage: 346KB (peak 365KB, 540KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 8.00e+01,  5.20e+02] / [ 8.00e+01,  5.50e+02]
  Objective      [min,max] : [ 3.00e+00,  4.50e+01] / [ 3.00e+00,  4.80e+01]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0       26649.99956      D      5     0        .000000     0
     7       103960.0000      D      0     0        .000000     0
Uncrunching matrix
Opti